In [1084]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [1085]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
⌃ [6e4b80f9] BenchmarkTools v1.5.0
  [f67ccb44] HDF5 v0.17.2
⌅ [9136182c] ITensors v0.6.16
  [682c06a0] JSON v0.21.4
⌃ [33e6dc65] MKL v0.7.0
⌃ [91a5bcdd] Plots v1.40.5
⌃ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.10
⌃ [a759f4b9] TimerOutputs v0.5.24
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [1086]:
# benchmark adder_MPO
using ITensors
include("CT/src/test_adder_MPO.jl")
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random
L = 4
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# @time adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];


[1, 2, 3, 4]
[1, 4, 2, 3]
[1, 3, 4, 2]


([0, 1, 0, 1], 5)

In [1087]:
initial_state = productMPS(qubit_site, [2,2,2,2])
include("global_adder_passthrough.jl")

global_adder_folded(initial_state, ram_phy, shift_1_3_bits)
# single_site_add_folded!(shift_1_3_bits[ram_phy[ram_pos]], ram_pos, initial_state)


Cut between sites 1 and 2: [1.0, 0.0]
Cut between sites 2 and 3: [1.0]
Cut between sites 3 and 4: [1.0]


MPS
[1] ((dim=2|id=524|"Qubit,Site,n=1"), (dim=2|id=596|"Link,l=1"))
[2] ((dim=2|id=647|"Qubit,Site,n=2"), (dim=2|id=596|"Link,l=1"), (dim=1|id=533|"Link,l=2"))
[3] ((dim=1|id=533|"Link,l=2"), (dim=2|id=846|"Qubit,Site,n=3"), (dim=1|id=695|"Link,l=3"))
[4] ((dim=1|id=695|"Link,l=3"), (dim=2|id=911|"Qubit,Site,n=4"))


In [1088]:
tensor = contract(initial_state)
print_nonzero_coordinates(tensor.tensor)

Nonzero at coordinates (1, 1, 2, 1): 1.0


In [1089]:
inds(initial_state[1])
orthogonalize!(initial_state, 1)

MPS
[1] ((dim=2|id=524|"Qubit,Site,n=1"), (dim=2|id=837|"Link,l=1"))
[2] ((dim=2|id=647|"Qubit,Site,n=2"), (dim=1|id=936|"Link,l=2"), (dim=2|id=837|"Link,l=1"))
[3] ((dim=2|id=846|"Qubit,Site,n=3"), (dim=1|id=927|"Link,l=3"), (dim=1|id=936|"Link,l=2"))
[4] ((dim=2|id=911|"Qubit,Site,n=4"), (dim=1|id=927|"Link,l=3"))


In [1090]:
initial_state[4].tensor

Dim 1: (dim=2|id=911|"Qubit,Site,n=4")
Dim 2: (dim=1|id=927|"Link,l=3")
NDTensors.Dense{Float64, Vector{Float64}}
 2×1
 1.0
 0.0

In [1091]:
tmp = initial_state[1] * initial_state[2]

ITensor ord=3 (dim=2|id=524|"Qubit,Site,n=1") (dim=2|id=647|"Qubit,Site,n=2") (dim=1|id=936|"Link,l=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [1092]:
inds(tmp)

((dim=2|id=524|"Qubit,Site,n=1"), (dim=2|id=647|"Qubit,Site,n=2"), (dim=1|id=936|"Link,l=2"))

In [1093]:
tmp.tensor[:,2,:,1,1]

Dim 1: 2
Dim 2: 1
NDTensors.Dense{Float64, Base.ReshapedArray{Float64, 1, SubArray{Float64, 2, Array{Float64, 3}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64, Base.Slice{Base.OneTo{Int64}}}, false}, Tuple{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64}}}}
 2×1
 0.0
 0.0

In [1094]:
single_site_add_folded!(shift_1_3_bits[ram_phy[2]], 2, initial_state)
println(shift_1_3_bits[ram_phy[2]])

1


In [1095]:
lsb_tensor = ITensor(filterinds(initial_state[2], tags="Carry,c=1"))
lsb_tensor[inds(lsb_tensor)[1]=>1] = 1.0
# print(inds(lsb_tensor))
initial_state[2] = initial_state[2] * lsb_tensor;
# println(inds(initial_state[2]))

# discard the carry out of the msb tensor
msb_tensor = ITensor(filterinds(initial_state[1], tags="Carry,c=0"))
msb_tensor[inds(msb_tensor)[1]=>1] = 1.0
msb_tensor[inds(msb_tensor)[1]=>2] = 1.0
initial_state[1] = initial_state[1] * msb_tensor;

LoadError: BoundsError: attempt to access Tuple{} at index [1]

In [ ]:
println(inds(initial_state[2]))

In [ ]:
tmp2 = initial_state[1] * initial_state[2] * initial_state[3]
println(inds(tmp2))

In [ ]:
size(tmp2)

In [ ]:
using ITensors

function print_nonzero_coordinates(T)
    # Get the dimensions of the tensor
    dims = size(T)
    
    # Iterate through all possible indices
    for idx in CartesianIndices(dims)
        # Check if element is nonzero
        if T[idx] != 0
            # Convert CartesianIndex to tuple for cleaner output
            coords = Tuple(idx)
            println("Nonzero at coordinates $coords: $(T[idx])")
        end
    end
end

print_nonzero_coordinates(tmp2.tensor)

In [ ]:
c2 = inds(initial_state[2])[2]
println(inds(initial_state[2]))
c4 = Index(2, "Carry,c=4")
T_Lm1 = create_addition_tensor_with_carry(1, qubit_site[L-1], prime(qubit_site[L-1]), c4, c2)

In [ ]:
tensor = contract(initial_state)
println(inds(tensor))

In [ ]:

# qubit_site = siteinds(initial_state)
# T_left_ind = Index(2, "Carry,c=$(ram_pos-1)")
# T_right_ind = Index(2, "Carry,c=$(ram_pos)")
# T = create_addition_tensor_with_carry(shift_bit, qubit_site[ram_pos], prime(qubit_site[ram_pos]), T_right_ind, T_left_ind);
# id_op_right_ind = Index(2, "Carry,c=$(ram_pos+1)")
# id_op = create_identity_tensor_4d(qubit_site[ram_pos+1], prime(qubit_site[ram_pos+1]), id_op_right_ind, T_right_ind);
# # println("T", inds(T))
# # println("id_op", inds(id_op))
# gate = T * id_op
# # figure out the left and right indices for the SVD
# left_original_inds = inds(initial_state[ram_pos]) # bookkeep some of the original indices from the left site for SVD
# link_right = filterinds(initial_state[ram_pos+1], tags="Link,l=$(ram_pos)")
# left_original_inds = setdiff(left_original_inds, link_right)
# # println("left_original_inds ", left_original_inds)

# # contract the gate with the two qubit sites
# psi = gate * (initial_state[ram_pos] * initial_state[ram_pos+1])
# noprime!(psi)
# carry_left = filterinds(psi, tags="Carry,c=$(ram_pos-1)")
# # println("carry_left", carry_left)
# link_left = filterinds(psi, tags="Link,l=$(ram_pos-1)")
# # println("link_left ", link_left)
# qubit_left = filterinds(psi, tags="Site,n=$(ram_pos)")
# # println(qubit_left)
# left_inds = unioninds(carry_left, link_left, qubit_left, left_original_inds)
# println("left_inds ", left_inds)
# # println(typeof(psi))
# U, S, V = svd(psi, left_inds; cutoff=1e-12, lefttags = "Link,l=$(ram_pos)")
# # println(inds(initial_state))
# # new_state = copy(initial_state)
# initial_state[ram_pos] = U
# initial_state[ram_pos+1] = S*V

In [ ]:
new_psi = initial_state[1] * initial_state[2]
println(inds(new_psi))
println(new_psi.tensor[:,1,1,:,1])

In [ ]:
psi = gate * (initial_state[1] * initial_state[2])

In [ ]:
psi.tensor[:,2,:,1,1]

In [ ]:
println(inds(initial_state[1]))

In [ ]:
typeof(initial_state[1])

In [ ]:
tensor1 = initial_state[1]
indices1 = inds(tensor1)
println(indices1)
link_num = 2
tensor1[indices1[1]=>2, indices1[2]=>2, indices1[3]=>link_num]

In [ ]:
tensor2 = initial_state[2]
indices2= inds(tensor2)
println(indices2)
tensor2[indices2[1] => 1, indices2[2]=>1, indices2[3]=>2, indices2[4]=>1]

In [ ]:
initial_state[2].tensor[2,:,:]

In [ ]:
# initial_state = productMPS(qubit_site, [1,2,2,1])
# shift_bit = 0
# ram_pos = 1
# include("global_adder_passthrough.jl")
# T_left_ind = Index(2, "Carry,c=$(ram_pos-1)")
# T_right_ind = Index(2, "Carry,c=$(ram_pos)")
# T = create_addition_tensor_with_carry(shift_bit, qubit_site[ram_pos], prime(qubit_site[ram_pos]), T_left_ind, T_right_ind);
# id_op_right_ind = Index(2, "Carry,c=$(ram_pos+1)")
# id_op = create_identity_tensor_4d(qubit_site[ram_pos+1], prime(qubit_site[ram_pos+1]), T_right_ind, id_op_right_ind);
# gate = T * id_op
# # figure out the left and right indices for the SVD
# left_original_inds = inds(initial_state[ram_pos]) # bookkeep some of the original indices from the left site for SVD
# link_right = filterinds(initial_state[ram_pos+1], tags="Link,l=$(ram_pos)")
# left_original_inds = setdiff(left_original_inds, link_right)
# # println("left_original_inds ", left_original_inds)

# # contract the gate with the two qubit sites
# psi = gate * (initial_state[ram_pos] * initial_state[ram_pos+1])
# noprime!(psi)
# carry_left = filterinds(psi, tags="Carry,c=$(ram_pos-1)")
# # println("carry_left", carry_left)
# link_left = filterinds(psi, tags="Link,l=$(ram_pos-1)")
# # println("link_left ", link_left)
# qubit_left = filterinds(psi, tags="Site,n=$(ram_pos)")
# # println(qubit_left)
# left_inds = unioninds(carry_left, link_left, qubit_left, left_original_inds)
# # println("left_inds ", left_inds)
# # println(typeof(psi))
# U, S, V = svd(psi, left_inds; cutoff=1e-12, lefttags = "Link,l=$(ram_pos)")
# # println(inds(initial_state))
# # new_state = copy(initial_state)
# initial_state[ram_pos] = U
# initial_state[ram_pos+1] = S*V

In [ ]:
# println(inds(initial_state))

# msb_tensor = ITensor(filterinds(initial_state[1], tags="Carry,c=0"))
# msb_tensor[inds(msb_tensor)[1]=>1] = 1.0
# msb_tensor[inds(msb_tensor)[1]=>2] = 1.0
# initial_state[1] = initial_state[1] * msb_tensor;
# normalize!(initial_state)

# println(initial_state)

In [ ]:
# L = 40
# ancilla = 0
# folded = true
# seed_vec = 123457
# xj = Set([1//3, 2//3])
# i1 = 1
# p_ctrl = 1.0
# p_proj = 0.0
# _maxdim = 50
# _cutoff = 1e-15
# seed = 123457
# x0 = nothing
# qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
# println(phy_list)
# println(ram_phy)
# println(phy_ram)
# rng = MersenneTwister(seed_vec)
# rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# # initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# # println(initial_state)
# shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# # @time adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L]



In [ ]:
# # Method 1: @time (shows time + memory allocation + GC time)
# @time adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];

# # Method 2: @allocated (only memory allocation in bytes)
# memory_allocated = @allocated adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];
# println("Memory allocated: $(memory_allocated / 1024^2) MB")

# # Method 3: Manual tracking with GC.gc()
# GC.gc()  # Force garbage collection
# mem_before = Base.gc_live_bytes()
# adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];
# GC.gc()  # Force garbage collection
# mem_after = Base.gc_live_bytes()
# println("Live memory change: $((mem_after - mem_before) / 1024^2) MB")


In [ ]:
# mps = _initialize_vector(L, ancilla, nothing, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim)
# @time apply(adder[3], mps)

In [ ]:
# # Method 6: System-level memory monitoring (Linux/Unix)
# # This tracks actual RSS (Resident Set Size) memory usage

# function get_memory_usage()
#     try
#         # Read from /proc/self/status on Linux
#         status = read("/proc/self/status", String)
#         for line in split(status, '\n')
#             if startswith(line, "VmRSS:")
#                 # Extract memory in kB and convert to MB
#                 memory_kb = parse(Int, split(line)[2])
#                 return memory_kb / 1024  # Convert to MB
#             end
#         end
#     catch
#         # Fallback: use Julia's garbage collector info
#         return Base.gc_live_bytes() / 1024^2
#     end
#     return 0.0
# end

# # Track memory before and after
# println("Memory before: $(get_memory_usage()) MB")
# @time adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];
# println("Memory after: $(get_memory_usage()) MB")
